In [ ]:
# # For GPU
# ! pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html
# # Restart Session
# # For CPU
! pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/repo.html
! pip install dgllife
! pip install rdkit
! pip install fvcore

Looking in links: https://data.dgl.ai/wheels/torch-2.4/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 12.8 MB/s eta 0:00:00
Discarding https://data.dgl.ai/wheels/torch-2.4/dgl-2.5.0-cp312-cp312-manylinux1_x86_64.whl (from https://data.dgl.ai/wheels/torch-2.4/repo.html): Requested dgl from https://data.dgl.ai/wheels/torch-2.4/dgl-2.5.0-cp312-cp312-manylinux1_x86_64.whl has inconsistent version: expected '2.5.0', but metadata has '2.5.0+cpu'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler

In [ ]:
FOLD = '2'

In [ ]:
train_df = pd.read_csv(f'/content/drive/MyDrive/Research_with_Atiq/PARP-1/Fold_{FOLD}/train.csv')
val_df = pd.read_csv(f'/content/drive/MyDrive/Research_with_Atiq/PARP-1/Fold_{FOLD}/val.csv')
test_df = pd.read_csv(f'/content/drive/MyDrive/Research_with_Atiq/PARP-1/Fold_{FOLD}/test.csv')

In [ ]:
def replacing0s(p_df):
    mn_con = float('inf')
    zero_indices = []
    for ii,con in enumerate(p_df['IC50']):
        if con <= 0.0:
            zero_indices.append(ii)
            continue
        mn_con = min(mn_con,con)
    for zero_index in zero_indices:
        p_df.loc[zero_index,'IC50'] = mn_con/2.0
        p_df.loc[zero_index,'pIC50'] = -np.log10((mn_con/2.0)*1e-9)
    return p_df

train_df = replacing0s(train_df)
val_df = replacing0s(val_df)
test_df = replacing0s(test_df)

In [ ]:
scaler = StandardScaler()
y = train_df[['pIC50']].values  # shape (N, 1)
y_scaled = scaler.fit_transform(y)

train_df['pIC50_scaled'] = y_scaled
val_df['pIC50_scaled'] = scaler.transform(val_df[['pIC50']].values)
test_df['pIC50_scaled'] = scaler.transform(test_df[['pIC50']].values)

In [ ]:
parp_log_df = train_df[['Smiles', 'IC50', 'pIC50', 'pIC50_scaled']].copy()
parp_log_val_df = val_df[['Smiles', 'IC50', 'pIC50', 'pIC50_scaled']].copy()
parp_log_test_df = test_df[['Smiles', 'IC50', 'pIC50', 'pIC50_scaled']].copy()

psic50_max = max(parp_log_df['pIC50_scaled'])
psic50_min = min(parp_log_df['pIC50_scaled'])

pvsic50_max = max(parp_log_val_df['pIC50_scaled'])
pvsic50_min = min(parp_log_val_df['pIC50_scaled'])

ptsic50_max = max(parp_log_test_df['pIC50_scaled'])
ptsic50_min = min(parp_log_test_df['pIC50_scaled'])

print(f'Train Max psIC50 {psic50_max} Min {psic50_min}, diff {psic50_max-psic50_min}')
print(f'Val Max psIC50 {pvsic50_max} Min {pvsic50_min}, diff {pvsic50_max-pvsic50_min}')
print(f'Test Max psIC50 {ptsic50_max} Min {ptsic50_min}, diff {ptsic50_max-ptsic50_min}')

Train Max psIC50 3.2552343902936802 Min -6.714566663133023, diff 9.969801053426703
Val Max psIC50 2.1993112166768944 Min -3.010272769725027, diff 5.209583986401921
Test Max psIC50 2.972480900033525 Min -3.561507737482061, diff 6.533988637515586


In [ ]:
# train_df
parp_train_df = train_df[['Smiles', 'pIC50_scaled']].copy()
parp_val_df = val_df[['Smiles', 'pIC50_scaled']].copy()
parp_test_df = test_df[['Smiles', 'pIC50_scaled']].copy()
parp_train_df.head()

,Smiles,pIC50_scaled
0,CNC(=O)c1ccc2c(n1)OC[C@@H]1CN(Cc3cnc4cc(C5CC5)...,0.905912
1,O=C(CCCc1nc2ccccc2c(=O)[nH]1)N[C@H]1CC[C@H](OC...,-1.604568
2,CCCN1CCN(C(=O)c2ccc3[nH]c(=O)c4c(c3c2)CCSC4)CC1,0.238616
3,CN(C)C1CCN(C(=O)c2cc(Cc3c[nH]c(=O)c4cc(Cl)c(Cl...,1.261085
4,O=C(c1cc(Cc2n[nH]c(=O)c3ccccc23)ccc1F)N1CCN(c2...,-6.714567


# PARPfusion

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

from dgllife.model.pretrain import load_pretrained as dgl_load_pretrained_model

In [ ]:
BATCH_SIZE = 4
EPOCHS = 20
LR = 5e-5
WD = 1e-5
IPV = 1
OF = f'/content/drive/MyDrive/Research_with_Atiq/PARP-1/outputs_{FOLD}'

In [ ]:
MD_universal = torch.load('/content/drive/MyDrive/Research_with_Atiq/PARP-1/universal_MD_tensor.pth')
CB_universal = torch.load('/content/drive/MyDrive/Research_with_Atiq/PARP-1/universal_chemberta_tensor.pth')

def checking_if_every_tensor_is_present(parp_df_list, md_univ, cb_univ):

    len_lst = len(parp_df_list)

    for ix in range(len_lst):
        for ii,smile in enumerate(parp_df_list[ix]['Smiles']):
            if md_univ.get(smile) is None:
                raise KeyError('MD universal doesnt have tensors for all the smile.')
            if cb_univ.get(smile) is None:
                raise KeyError('MD universal doesnt have tensors for all the smile.')


checking_if_every_tensor_is_present(
    parp_df_list = [parp_train_df, parp_val_df, parp_test_df],
    md_univ = MD_universal,
    cb_univ = CB_universal
)

/tmp/ipython-input-1142700921.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  MD_universal = torch.load('/content/drive/MyDrive/Research_with_Atiq/PARP-1/universal_MD_te

In [ ]:
len(MD_universal)

3356

In [ ]:
from dgllife.data.csv_dataset import MoleculeCSVDataset
from dgllife.utils.featurizers import CanonicalAtomFeaturizer, CanonicalBondFeaturizer, AttentiveFPAtomFeaturizer, AttentiveFPBondFeaturizer
from dgllife.utils.mol_to_graph import SMILESToBigraph
from dgl.dataloading import GraphDataLoader
import dgl

smiles_to_graph_can = SMILESToBigraph(
    add_self_loop=True,
    node_featurizer=CanonicalAtomFeaturizer(),
    edge_featurizer=CanonicalBondFeaturizer(self_loop=True)
)

smiles_to_graph_fp = SMILESToBigraph(
    add_self_loop=True,
    node_featurizer=AttentiveFPAtomFeaturizer(),
    edge_featurizer=AttentiveFPBondFeaturizer(self_loop=True)
)

can_parp_train_dataset = MoleculeCSVDataset(
    df = parp_train_df,
    smiles_to_graph = smiles_to_graph_can,
    smiles_column = 'Smiles',
    cache_file_path = '/content/drive/MyDrive/Research_with_Atiq/PARP-1/cache_dglgraphs_train_can.bin'
)

can_parp_val_dataset = MoleculeCSVDataset(
    df = parp_val_df,
    smiles_to_graph = smiles_to_graph_can,
    smiles_column = 'Smiles',
    cache_file_path = '/content/drive/MyDrive/Research_with_Atiq/PARP-1/cache_dglgraphs_val_can.bin'
)

can_parp_test_dataset = MoleculeCSVDataset(
    df = parp_test_df,
    smiles_to_graph = smiles_to_graph_can,
    smiles_column = 'Smiles',
    cache_file_path = '/content/drive/MyDrive/Research_with_Atiq/PARP-1/cache_dglgraphs_test_can.bin'
)

fp_parp_train_dataset = MoleculeCSVDataset(
    df = parp_train_df,
    smiles_to_graph = smiles_to_graph_fp,
    smiles_column = 'Smiles',
    cache_file_path = '/content/drive/MyDrive/Research_with_Atiq/PARP-1/cache_dglgraphs_train_fp.bin'
)

fp_parp_val_dataset = MoleculeCSVDataset(
    df = parp_val_df,
    smiles_to_graph = smiles_to_graph_fp,
    smiles_column = 'Smiles',
    cache_file_path = '/content/drive/MyDrive/Research_with_Atiq/PARP-1/cache_dglgraphs_val_fp.bin'
)

fp_parp_test_dataset = MoleculeCSVDataset(
    df = parp_test_df,
    smiles_to_graph = smiles_to_graph_fp,
    smiles_column = 'Smiles',
    cache_file_path = '/content/drive/MyDrive/Research_with_Atiq/PARP-1/cache_dglgraphs_test_fp.bin'
)

can_train_loader = GraphDataLoader(can_parp_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
can_val_loader = GraphDataLoader(can_parp_val_dataset, batch_size=BATCH_SIZE, shuffle=False)
can_test_loader = GraphDataLoader(can_parp_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

fp_train_loader = GraphDataLoader(fp_parp_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
fp_val_loader = GraphDataLoader(fp_parp_val_dataset, batch_size=BATCH_SIZE, shuffle=False)
fp_test_loader = GraphDataLoader(fp_parp_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Processing dgl graphs from scratch...
Processing molecule 1000/2516
Processing molecule 2000/2516
Processing dgl graphs from scratch...
Processing dgl graphs from scratch...
Processing dgl graphs from scratch...
Processing molecule 1000/2516
Processing molecule 2000/2516
Processing dgl graphs from scratch...
Processing dgl graphs from scratch...


In [ ]:
class IC50Regressor(nn.Module):
    def __init__(self, input_dim=774):
        super(IC50Regressor, self).__init__()

        self.fc1 = nn.Linear(input_dim, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 1)

        self.dropout = nn.Dropout(0.3)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.dropout(self.act(self.bn1(self.fc1(x))))
        x = self.dropout(self.act(self.bn2(self.fc2(x))))
        return self.fc3(x)  # linear output (no activation)

class PARPfusion(nn.Module):
    def __init__(self, gcn, reg_in_size = 774):
        super(PARPfusion, self).__init__()

        self.gnn = gcn.gnn
        self.readout = gcn.readout
        self.predict = IC50Regressor(input_dim = reg_in_size)

    def forward(self, X):

        bg, cb, md = X

        node_feats = self.gnn(bg, bg.ndata['h'])
        graph_feats = self.readout(bg,node_feats)

        check_tensors((graph_feats))

        md = (md - md.mean(dim=0)) / (md.std(dim=0) + 1e-6)
        cb = (cb - cb.mean(dim=0)) / (cb.std(dim=0) + 1e-6)
        graph_feats = (graph_feats - graph_feats.mean(dim=0)) / (graph_feats.std(dim=0) + 1e-6)

        x = torch.cat([graph_feats, cb, md], dim=1)
        return self.predict(x)

class PARPfusionAblation(PARPfusion):
    def __init__(self, gcn, ablation_conf=7):

        self.ablation_conf = ablation_conf
        reg_in_sizes = [134, 384, 256, 518, 640, 390, 774]
        super().__init__(
            gcn = gcn,
            reg_in_size = reg_in_sizes[self.ablation_conf-1]
        )


    def forward(self, X):
        bg, cb, md = X

        if self.ablation_conf >= 3:
            node_feats = self.gnn(bg, bg.ndata['h'])
            graph_feats = self.readout(bg,node_feats)

            check_tensors((graph_feats))


            graph_feats = (graph_feats - graph_feats.mean(dim=0)) / (graph_feats.std(dim=0) + 1e-6) # 256

        md = (md - md.mean(dim=0)) / (md.std(dim=0) + 1e-6) # 134
        cb = (cb - cb.mean(dim=0)) / (cb.std(dim=0) + 1e-6) # 384

        if self.ablation_conf == 1:
            return self.predict(md)
        elif self.ablation_conf == 2:
            return self.predict(cb)
        elif self.ablation_conf == 3:
            return self.predict(graph_feats)
        elif self.ablation_conf == 4:
            x = torch.cat([md,cb], dim=1)
            return self.predict(x)
        elif self.ablation_conf == 5:
            x = torch.cat([cb,graph_feats], dim=1)
            return self.predict(x)
        elif self.ablation_conf == 6:
            x = torch.cat([md,graph_feats], dim=1)
            return self.predict(x)
        elif self.ablation_conf == 7:
            x = torch.cat([md,cb,graph_feats], dim=1)
            return self.predict(x)



In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr, spearmanr
import json
import os
from fvcore.nn import FlopCountAnalysis
import time
from functools import wraps

def gen_scores(trues, preds):
    mse = mean_squared_error(trues, preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(trues, preds)
    r2 = r2_score(trues, preds)
    pearson_corr, _ = pearsonr(trues, preds)
    spearman_corr, _ = spearmanr(trues, preds)

    results = {
        "MSE": float(mse),
        "RMSE": float(rmse),
        "MAE": float(mae),
        "R^2": float(r2),
        "Pearson": float(pearson_corr),
        "Spearman": float(spearman_corr)
    }

    return results

def save_json(dd, file_path):
    with open(file_path, "w") as f:
        json.dump(dd, f, indent=4)

def check_model_req_grad(model):
    model_desc = []
    for name, param in model.named_parameters():
        model_desc.append(f'{name}, {param.requires_grad}\n')
    return model_desc

def check_tensors(t_tup):
    updated_tup = []
    for tt in t_tup:
        if torch.isnan(tt).any() or torch.isinf(tt).any():
            tt = torch.nan_to_num(tt, nan=0.0, posinf=0.0, neginf=0.0)
        updated_tup.append(tt)
    return tuple(updated_tup)

def estimate_flop_count_gcn(model, inp):
    bgs, cbs, mds = inp
    N = bgs.num_nodes()
    E = bgs.num_edges()

    model_desc = check_model_req_grad(model)

    num_gcn_layers = 0
    for ss in model_desc:
        if 'graph_conv.weight' in ss:
            num_gcn_layers += 1

    in_dim, hidden_dim, out_dim = 74, 128, 256

    flops_per_layer = 2 * E * in_dim + N * in_dim * hidden_dim
    gcn_total_flops = flops_per_layer * (num_gcn_layers - 1)

    # output layer (no message passing)
    gcn_total_flops += N * hidden_dim * out_dim

    linear_total_flops = FlopCountAnalysis(model.predict, torch.randn(4,774)).total()
    print(gcn_total_flops, linear_total_flops)
    return gcn_total_flops + linear_total_flops

def measure_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        elapsed = end - start

        if args and hasattr(args[0], 'inf_time'):
            args[0].inf_time = elapsed
        else:
            print(f"{func.__name__} took {elapsed:.6f} seconds")

        return result
    return wrapper

def count_trainable_param(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def get_saved_tensors(smiles):
    len_smiles = len(smiles)
    mds = torch.stack([MD_universal[smiles[batch_index]] for batch_index in range(len_smiles)], dim=0).float()
    cbs = torch.stack([CB_universal[smiles[batch_index]] for batch_index in range(len_smiles)], dim=0).float()
    return mds, cbs

In [ ]:
class BAPTrainer():
    def __init__(
            self,
            model_name,
            train_loader,
            val_loader,
            test_loader,
            batch_size,
            epochs,
            lr,
            weight_decay,
            itr_per_val,
            output_folder
        ):
        self.model_name = model_name
        self.model = self.init_model()

        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader

        self.criterion = nn.MSELoss()
        self.optimizer = Adam(self.model.parameters(), lr=lr, weight_decay=weight_decay)
        self.scheduler = CosineAnnealingLR(self.optimizer, T_max=epochs)

        self.batch_size = batch_size
        self.epochs = epochs
        self.itr_per_val = itr_per_val
        self.output_folder = output_folder

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.inf_time = -1.0
        self.train_loss  = []
        self.val_loss = []
        self.best_val_dict = None
        self.test_score_dict = None

    def init_model(self):
        if self.model_name == 'parp_fusion_canonical':
            gcn = dgl_load_pretrained_model(model_name = 'GCN_canonical_BACE')
            return PARPfusion(gcn=gcn)
        elif self.model_name == 'parp_fusion_attentivefp':
            gcn = dgl_load_pretrained_model(model_name = 'GCN_attentivefp_BACE')
            return PARPfusion(gcn=gcn, reg_in_size=64
                              6)
        elif 'pf_can_ab_' in self.model_name:
            gcn = dgl_load_pretrained_model(model_name = 'GCN_canonical_BACE')
            return PARPfusionAblation(gcn=gcn, ablation_conf=int(self.model_name.split('_')[-1]))
        else:
            return dgl_load_pretrained_model(model_name = self.model_name)

    def forward_pass(self,inp):
        if "GAT" in self.model_name:
            bgs, cbs, mds = inp
            return self.model(bgs, bgs.ndata['h'])
        elif self.model_name.split('_')[0] in ["Weave", "MPNN", "AttentiveFP"]:
            bgs, cbs, mds = inp
            return self.model(bgs, bgs.ndata['h'], bgs.edata['e'])
        return self.model(inp)

    def val(self, d_loader, msg='Validating'):
        self.model.eval()
        preds, trues = [], []

        with torch.no_grad():
            for batch in tqdm(d_loader, desc=msg):
                smiles, bgs, labels, masks = batch
                bgs = bgs.to(self.device)
                labels = labels.to(self.device)

                mds, cbs = get_saved_tensors(smiles)
                mds, cbs, labels = check_tensors((mds, cbs, labels))

                outputs = self.forward_pass((bgs, cbs, mds))
                preds.append(outputs.squeeze().cpu().numpy())
                trues.append(labels.squeeze().cpu().numpy())

        preds = np.concatenate(preds)
        trues = np.concatenate(trues)

        preds = scaler.inverse_transform(preds.reshape(-1, 1)).flatten()
        trues = scaler.inverse_transform(trues.reshape(-1, 1)).flatten()

        return gen_scores(trues, preds)

    @measure_time
    def test(self):
        self.model.load_state_dict(torch.load(os.path.join(self.output_folder,f"best_model_{self.model_name}.pt")))
        scores = self.val(self.test_loader,'Testing')
        print(scores)
        self.test_score_dict = scores

    def train(self):
        self.model.train()
        best_val_mse = float('inf')
        for epc in range(self.epochs):
            epoch_loss = 0.0
            for g_batch in tqdm(self.train_loader, desc="Training batches"):
                smiles, bgs, labels, masks = g_batch

                mds, cbs = get_saved_tensors(smiles)
                mds, cbs, labels = check_tensors((mds, cbs, labels))

                if torch.isnan(mds).any() or torch.isinf(mds).any():
                    raise ValueError('Found NAN/INF in MDs')

                out = self.forward_pass((bgs, cbs, mds))

                loss = self.criterion(out, labels)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

                epoch_loss += loss.item()


            self.scheduler.step()
            print(f"Epoch {epc+1}, Loss: {epoch_loss/len(self.train_loader):.4f}")
            self.train_loss.append(float(epoch_loss/len(self.train_loader)))

            if epc % self.itr_per_val == 0:
                scores = self.val(self.val_loader)
                self.val_loss.append(scores['MSE'])
                if scores['MSE'] < best_val_mse:
                    best_val_mse = scores['MSE']
                    print(f'Found new best val mse at {epc+1}')
                    print(scores)
                    torch.save(self.model.state_dict(), os.path.join(self.output_folder,f"best_model_{self.model_name}.pt"))
                    self.best_val_dict = scores

    def saving_gen_info(self):
        dd = {
            "inf_seconds_per_sample": (self.inf_time/(len(self.test_loader)*self.batch_size)),
            "num_trainable_params": count_trainable_param(self.model),
            "train_loss": self.train_loss,
            "val_loss": self.val_loss,
            "best_val_score": self.best_val_dict,
            "test_score": self.test_score_dict
        }
        save_json(dd,os.path.join(self.output_folder,f'gen_info_{self.model_name}.json'))


In [ ]:
mns = [
    'parp_fusion_attentivefp',
    'GAT_attentivefp_BACE',
    'Weave_attentivefp_BACE',
    'MPNN_attentivefp_BACE',
    'AttentiveFP_attentivefp_BACE'
    ]
for mn in mns:
    bap = BAPTrainer(
        model_name = mn,
        train_loader = fp_train_loader,
        val_loader = fp_val_loader,
        test_loader = fp_test_loader,
        batch_size = BATCH_SIZE,
        epochs = EPOCHS,
        lr = LR,
        weight_decay = WD,
        itr_per_val = IPV,
        output_folder = OF
    )
    bap.train()
    bap.test()
    bap.saving_gen_info()

NameError: name 'BAPTrainer' is not defined

In [ ]:
mns = [
    'parp_fusion_canonical',
    'GAT_canonical_BACE',
    'Weave_canonical_BACE',
    'MPNN_canonical_BACE',
    'AttentiveFP_canonical_BACE'
    ]
for mn in mns:
    bap = BAPTrainer(
        model_name = mn,
        train_loader = can_train_loader,
        val_loader = can_val_loader,
        test_loader = can_test_loader,
        batch_size = BATCH_SIZE,
        epochs = EPOCHS,
        lr = LR,
        weight_decay = WD,
        itr_per_val = IPV,
        output_folder = OF
    )
    bap.train()
    bap.test()
    bap.saving_gen_info()

GCN_canonical_BACE_pre_trained.pth:   0%|          | 0.00/100k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/dgllife/model/pretrain/__init__.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(local_pretrained_path, m

Pretrained model loaded


Training batches: 100%|██████████| 629/629 [00:10<00:00, 60.27it/s]


Epoch 1, Loss: 0.9072


...

Epoch 20, Loss: 0.7817


Validating: 100%|██████████| 42/42 [00:00<00:00, 61.49it/s]
/tmp/ipython-input-2218266806.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(tor

Found new best val mse at 20
{'MSE': 1.0863977670669556, 'RMSE': 1.042304066511762, 'MAE': 0.8143417835235596, 'R^2': 0.18047809600830078, 'Pearson': 0.4275232255458832, 'Spearman': 0.41910909600460583}


Testing: 100%|██████████| 168/168 [00:02<00:00, 76.17it/s]

{'MSE': 1.3015902042388916, 'RMSE': 1.1408725626637235, 'MAE': 0.8718022108078003, 'R^2': 0.16700494289398193, 'Pearson': 0.4123709499835968, 'Spearman': 0.38283396332580233}


# Ablation Study

In [ ]:
mns = [
    'pf_can_ab_1',
    'pf_can_ab_2',
    'pf_can_ab_3',
    'pf_can_ab_4',
    'pf_can_ab_5',
    'pf_can_ab_6',
    'pf_can_ab_7'
    ]
for mn in mns:
    bap = BAPTrainer(
        model_name = mn,
        train_loader = can_train_loader,
        val_loader = can_val_loader,
        test_loader = can_test_loader,
        batch_size = BATCH_SIZE,
        epochs = EPOCHS,
        lr = LR,
        weight_decay = WD,
        itr_per_val = IPV,
        output_folder = OF
    )
    bap.train()
    bap.test()
    bap.saving_gen_info()

GCN_canonical_BACE_pre_trained.pth:   0%|          | 0.00/100k [00:00<?, ?B/s]

Pretrained model loaded


Training batches: 100%|██████████| 629/629 [00:05<00:00, 125.43it/s]


Epoch 1, Loss: 0.9713


Validating: 100%|██████████| 42/42 [00:00<00:00, 593.59it/s]


...

Epoch 19, Loss: 0.2527


Training batches: 100%|██████████| 629/629 [00:11<00:00, 54.81it/s]


Epoch 20, Loss: 0.2689


Validating: 100%|██████████| 42/42 [00:00<00:00, 235.93it/s]
/tmp/ipython-input-3721708315.py:87: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(to

{'MSE': 0.8408711552619934, 'RMSE': 0.9169902699930863, 'MAE': 0.6877325177192688, 'R^2': 0.45428764820098877, 'Pearson': 0.6756922602653503, 'Spearman': 0.6843909151243678}


# 95 Conf Score Gen

In [ ]:
import scipy.stats as stats
import numpy as np

def generateConfidenceScore(model_name):

    results = []

    for ii in range(5):
        json_path = f'/content/drive/MyDrive/Research_with_Atiq/PARP-1/outputs_{ii+1}/gen_info_{model_name}.json'
        with open(json_path, 'r') as f:
            dd_sc = json.load(f)
            if dd_sc.get('test_score') != None:
                results.append(dd_sc['test_score'])
            else:
                results.append(dd_sc)

    score_dict = {}

    for dd in results:
        for kk in dd:
            if kk == 'Model': continue
            if score_dict.get(kk) == None:
                score_dict[kk] = [dd[kk]]
            else:
                score_dict[kk].append(dd[kk])

    metric_conf_score = {}

    for kk in results[0]:
        if kk == 'Model':continue
        mean = np.mean(score_dict[kk])
        sem = stats.sem(score_dict[kk])
        confidence = 0.95
        n = len(score_dict[kk])
        df = n - 1
        t_critical = stats.t.ppf((1 + confidence) / 2, df)
        margin_of_error = t_critical * sem
        metric_conf_score[kk] = {
            'mean': mean,
            'margin_of_error': margin_of_error
        }

    return metric_conf_score

In [ ]:
mns = [
    'parp_fusion_attentivefp',
    'GAT_attentivefp_BACE',
    'Weave_attentivefp_BACE',
    'MPNN_attentivefp_BACE',
    'AttentiveFP_attentivefp_BACE',
    'parp_fusion_canonical',
    'GAT_canonical_BACE',
    'Weave_canonical_BACE',
    'MPNN_canonical_BACE',
    'AttentiveFP_canonical_BACE',
    'RandomForest',
    'XGBoost',
    'KNN'
    ]
conf_score = {}
for mn in mns:
    metric_conf_score = generateConfidenceScore(mn)
    conf_score[mn] = metric_conf_score

save_json(
    conf_score,
    '/content/drive/MyDrive/Research_with_Atiq/PARP-1/conf_score_95.json'
)

